In [ ]:
# Por si alguien corre en python2
from __future__ import division

# Preparamos todo para correr
import numpy as np
from math import *
from matplotlib import pylab as plt
%matplotlib inline

## Mixturas Gaussianas
<a id='mixtures'></a>

Cuando estudiamos la distribución normal, vimos que uno de las ventajas de sus propiedades analíticas es que sirven como elementos de modelos más complejos. 

Una técnica para hacer esto es la mixtura de Gaussianas, que nos permite captar mucha más complejidad que una simple normal.

In [ ]:
# Si están en el colab, primero corran este código
!mkdir datasets
!wget https://raw.githubusercontent.com/exord/UNSAM_IA/master/datasets/faithful.csv
!mv faithful.csv datasets/

Estos datos contienen información sobre la erupciones de geisers en Old Faithful, en el parque [Yellowstone](https://www.nps.gov/yell/learn/photosmultimedia/webcams.htm).

Los datos son la duración de las erupciones en segundos, y el tiempo entre una erupción y la siguiente.

In [ ]:
# Leamos y miremos datos de Faithful
import pandas as pd
df = pd.read_csv('datasets/faithful.csv')

# Vamos las primeras columnas
df.head()

Vemos que hay una columna inutil, que nos vamos a sacar de encima y después veamos los datos.

In [ ]:
trash = df.pop('Unnamed: 0')

In [ ]:
l0 = df.plot('eruptions', 'waiting', kind='scatter')

Claramente, no podremos ajustar esto con una normal.
Calculemos los valores de máxima verosimilitud para $\mu$ y $\Sigma$ usando los datos y veamos qué tal da.

Recordemos los resultados

$$
\mathbf{\mu}_\mathrm{ML} = \frac{1}{N}\sum_{n=1}^{N}\mathbf{x}_n \\
$$
$$
\mathbf{\Sigma}_\mathrm{ML} = \frac{1}{N}\sum_{n=1}^{N} (\mathbf{x}_n - \mathbf{\mu}_\mathrm{ML})(\mathbf{x}_n - \mathbf{\mu}_\mathrm{ML})^T
$$

Calculemos eso para los datos de Old Faithful

In [ ]:
mu_ml = df.mean().to_numpy()

dx = (df - mu_ml)
sigma_ml = np.dot(dx.T, dx) / len(dx)

print(u'$\mu_ML$ = {}'.format(mu_ml))
print(u'$\Sigma_ML$ = {}'.format(sigma_ml))

Con estos valores, construyamos una binormal. Como no vamos a sacar valores al azar, usamos la implementación en <tt> scipy.stats</tt>.

In [ ]:
import scipy.stats as st
p = st.multivariate_normal(mean=mu_ml, cov=sigma_ml)

Ahora viene la parte divertida. Vamos a graficar los contornos de la binormal sobre los datos de Old Faithful.

In [ ]:
# Primero construyo una grilla de valores
x, y = np.mgrid[1.0:6.0:0.1, 40:100:1]
pos = np.dstack((x, y))

# Prepara figure
fig = plt.figure()
ax = fig.add_subplot(111)

# Vuelve a graficar datos grafica contornos (elegimos usar 4 contornos)
df.plot('eruptions', 'waiting', kind='scatter', ax=ax)
l0 = ax.contour(x, y, p.pdf(pos), 4, colors='0.5')

Claramente el modelo de una Gaussiana no es bueno. El modelo no solo no recoje el comportamiento bimodal de los datos, sino que además contiene mucha densidad en el centro entre ambos grupos de puntos.

Podemos calcular la verosimulitud de estos puntos. Suponiendo que son independientes:

$$
p(D | \mu_\mathrm{ML}, \Sigma_\mathrm{ML}) = \prod_{n=1}^N \mathcal{N(\mathbf{x}_n | \mu_\mathrm{ML}, \Sigma_\mathrm{ML}})\;\;.
$$

Evaluo entonces la densidad de la distribución en cada punto y multiplico. Para eso, usamos la práctica función <tt>apply<\tt> de Pandas

In [ ]:
densities = df.apply(p.pdf, axis=1)
print('La verosimilitud de este modelo es {}'.format(np.prod(densities)))

¿Qué pasó? Las densidades no son cero. Veamos:

In [ ]:
densities.head(10)

¿Alguna idea?

***

Nos encontramos con un problema típico cuando tratamos con muchos datos. La verosimilitud de cada uno de ellos es baja, y al multiplicarlos, llegamos por debajo del número más pequeño que puede representar la computadora. Este problema se hará más agudo cuantos más datos tengamos y cuanto mayor sea la dimensionalidad del espacio.

La solución es trabajar con el logaritmo de la verosimilitud.

En ese caso:

$$
\ln p(D | \mu_\mathrm{ML}, \Sigma_\mathrm{ML}) = \sum_{n=1}^N \ln \mathcal{N(\mathbf{x}_n | \mu_\mathrm{ML}, \Sigma_\mathrm{ML}}) = -\frac{1}{2}\sum_{n=1}^{n} \left[\ln(|\mathbf{\Sigma}|) + \ln(2\pi) + (\mathbf{x}_n - \mathbf{\mu}_\mathrm{ML})^T \mathbf{\Sigma}^{-1}(\mathbf{x}_n - \mathbf{\mu}_\mathrm{ML})\right]\;\;.
$$

Por suerte esto está implementado en <tt>scipy.stats</tt>.


In [ ]:
logdensities = df.apply(p.logpdf, axis=1)
print(logdensities.head())

In [ ]:
# Ahora solo hay que sumar
lnlike = np.sum(logdensities)
print(lnlike)

In [ ]:
# Fíjense que podríamos haber intentado directamente hacer el log de las densidades que ya teníamos; pero miren qué pasa
lnlike2 = np.sum(np.log(densities))
print(lnlike2)

***
### Presentación del modelo

Este ejemplo muestra una de las limitaciones de un modelo normal: solo puede captar distribuciones con un único modo. Una forma de sobreponerse a este problema es usar un modelo que consista en una superposición de $K$ distribuciones normales.

$$
p(\mathbf{x}) = \sum_{k=1}^K \pi_k{\mathcal{N}(\mathbf{x} \:|\: \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)}\;\;,
$$
donde cada densidad $\mathcal{N}(\mathbf{x} \:|\: \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)$ se conoce como una *componente* de la mixtura, y tiene su propio valor medio y matriz de covarianza. Los $\pi_k$ son los *coeficientes* de mezcla.

__Los coeficientes como probabilidades__

Integrando a ambos lados de la igualdad sobre todos los valore de $\mathbf{x}$ y viendo que tanto $p(\mathbf{x})$ como las componentes normales son funciones de distribución de probabilidad, llegamos a que

$$
\sum_{k=1}^K \pi_k = 1\;\;.
$$

Además, como $p(\mathbf{x})$ tienen que ser positiva, encontramos que una forma de garantizar eso de manera general es definir que los coeficientes sean positivos. Por lo tanto, $0 \leq \pi_k \leq 1$, y los coeficientes satisfacen los requerimientos para ser probabilidades.

Podemos, entonces, asignar una función de distribución de masa, $p(k)$, que describe la probabilidad (prior) de que se elija la compoenente $k$, y $p(k) = \pi_k$. Entonces, distribución sobre $\mathbf{x}$ queda:

$$
p(\mathbf{x}) = \sum_{k=1}^K p(\mathbf{x}, k) = \sum_{k=1}^K p(k) p(\mathbf{x} | k)\;\;,
$$

donde la probabilidad condicionada $p(\mathbf{x} | k) = \mathcal{N}(\mathbf{x} \:|\: \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)$.

En otras palabras, visto de esta manera, para muestrear el modelo de mixtura puede pensarse como una cadena, donde uno decide primero a qué modo pertence un punto y luego muestrea ese modo. Esto se llama muestreo ancestral. Veámoslo en acción.

In [ ]:
# Supongamos una distribución unidimensional de dos modos, con los siguientes parámetros
mu1 = [0.0,]
sigma1 = [2.0,]
mu2 = [5.0,]
sigma2 = [1.5,]

# Generemos las dos distris
n1 = st.norm(mu1, sigma1)
n2 = st.norm(mu2, sigma2)

# Ahora pensemos que tenemos una distribución prior p(k) con las siguientes elementos
p1 = 0.65
p2 = 0.35

# Veamos como podemos generar, fácilmente, muestras de la mixtura:
# p1 * N1 + p2 * N2
# Primero muestreo p(k)
N = 5000
k = np.where(np.random.rand(N) < p1, 1, 2)

# Y ahora lo combino con las mixturas

# Modo 1 (estricto)
# x1 = n1.rvs(sum(k==1), 1)
# x2 = n2.rvs(sum(k==2), 1)

# x = np.hstack([x1, x2])

# Modo 2 (generando de más)
x1 = n1.rvs(N, 1)
x2 = n2.rvs(N, 1)

x = np.where(k==1, x1, x2).reshape((-1, 1))

h = plt.hist(x, 50, histtype='step', density=True)

# Combinemos esto con la pdf del modelo
xx = np.linspace(x.min(), x.max(), 400)
plt.plot(xx, p1 * n1.pdf(xx) + p2 * n2.pdf(xx), 'r-')

In [ ]:
# Obviamente, como sabemos de dónde viene cada muestra, las podemos plotear por separado.
h = plt.hist(x, h[1], histtype='step', lw=5, color='k', alpha=0.2)
h1 = plt.hist(x[k==1], h[1], label='Componente 1', histtype='step')
h2 = plt.hist(x[k==2], h[1], label='Componente 2', histtype='step')
plt.legend(loc=0)

Pero mantengamos en mente que en los problemas que vamos a resolver, contaremos con la distribución azul de la celda anterior y sin información sobre el origen de cada muestra. Es como si hubiéramos perdido el arreglo $\mathbf{k}$. Al conjunto de datos $\{\mathbf{x}, \mathbf{k}\}$ se lo llama dataset completo, y a $\mathbf{x}$ se lo conoce como dataset incompleto. 

Claro que nunca estuvo completo, en principio....

__Responsabilidades__

Un rol crucial lo van a jugar las posteriores de la probabilidad de cada modo $p(k \:|\: \mathbf{x})$, que se conocen como *responsabilidades* (¡cuánta seriedad!)

$$
\gamma_k(\mathbf{x}) \equiv p(k \:|\: \mathbf{x})\;\;.
$$

Como es de esperar, podemos calcular la punta que tienen las responsabilidades a partir del teorema de Bayes:

$$
\begin{eqnarray}
\gamma_k(\mathbf{x}) &=& \frac{p(k) p(\mathbf{x}\:|\:k)}{\sum_i p(i) p(\mathbf{x}\:|\:i)}\\
   &=&\frac{\pi_k{\mathcal{N}(\mathbf{x} \:|\: \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)}}{\sum_i{\pi_i{\mathcal{N}(\mathbf{x} \:|\: \boldsymbol{\mu}_i, \boldsymbol{\Sigma}_i)}}}\;\;,
\end{eqnarray}
$$
donde usamos la definición de arriba
$$
p(\mathbf{x}) = \sum_{k=1}^K \pi_k{\mathcal{N}(\mathbf{x} \:|\: \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)}\;\;.
$$

__Verosimilitud__ (o Donde arrancan los líos)

Naturalmente, necesitaremos poder escribir la verosimilitud para un modelo de este tipo. Como vimos arriba, el problema con las exponenciales de la normal, es que muchas veces generan problemas numéricos.

Vamos entonces a escribir el logaritmo de la verosimilitud, como ya hicimos un montón de veces.

$$
\ln p(\mathbf{X} \:|\: \boldsymbol{\mu}, \boldsymbol{\Sigma}, \boldsymbol{\pi}) = \sum_{n=1}^N \ln \left\{\sum_{k=1}^K \pi_k{\mathcal{N}(\mathbf{x}_n \:|\: \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)}\right\}\;\;,
$$
donde estamos usando esta notación: $\boldsymbol{\mu} \equiv \left\{\boldsymbol{\mu}_1, \ldots, \boldsymbol{\mu}_K\right\}$, y lo mismo para $\boldsymbol{\Sigma}$ y para $\boldsymbol{\pi}$. Y como siempre $\mathbf{X} = \left\{\mathbf{x}_1, \ldots, \mathbf{x}_N\right\}$.

Vemos que el logaritmo actúa sobre la sumatoria y no sobre cada normal de manera individual. Esto hace que sea complicado obtener una forma cerrada para los parámetros que maximizan la verosimilitud. Tenemos que usar un procedimiento iterativo, conocido como el algoritmo de maximización de la expectación, o *expectation maximisation* (EM).


__Variables latentes / ocultas__

Para presentar de forma correcta el EM, primero tenemos que introducir un conjunto de nuevas variables, llamadas latentes. Más que un deseo de rigurosidad insaciable, nos mueve el hecho de que los modelos que tienen variables latentes suelen ser capaces de captar mucha complejidad de forma sencilla, y resultan una herramienta fundamental a la hora de modelar datos de la vida real. (**NB**: diría que, más allá de cualquier algoritmo particular que se puedan llevar de la materia, el concepto de las variables latentes y el truco del kernel son las dos herramientas más poderosas que presentamos; son como el martillo y el serrucho de un constructor de modelos). Vamos a ver enseguida como, agrandar el problema y pensar en una distribución conjunta facilita la resolución del problema condicional, $p(\mathbf{x} \:|\: k)$, que presentamos más arriba.

Vamos a introducir una variable latente binaria, $K$-dimensional, $z$, con una representación 1-de-K. Es decir, todos los elementos son 0, excepto por el elemento $k$ que es uno y que indica a qué modo pertenece una determinada observación. Claramente, se verifica que $\sum_k z_k = 1$ (la suma de los elementos de la variable $z$ es uno).

Entonces, la relación con los coeficientes anteriores es:

$$
p(z_k = 1) = \pi_k\;\;,
$$
y podemos escribir la distribución del _vector_ $\mathbf{z}$ como

$$
p(\mathbf{z}) = \prod_{k=1}^K \pi_k^{z_k}\;\;.
$$

Además, la condicional

$$
p(\mathbf{x} \:|\: z_k = 1) = \mathcal{N}(\mathbf{x} \:|\: \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)
$$
también puede escribirse para el vector $\mathbf{z}$ de la misa manera
$$
p(\mathbf{x} \:|\: \mathbf{z}) = \prod_{k=1}^K \mathcal{N}(\mathbf{x} \:|\: \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)^{z_k}\;\;.
$$

Bien, podemos entonces, plantear el problema original en términos de las distribuciones conjuntas, condicionales y marginales, $p(\mathbf{x}, \mathbf{z})$, $p(\mathbf{x} \:|\: \mathbf{z})$ y $p(\mathbf{z})$, que, como sabemos desde la primera clase, se relacionan a través de la regla del producto:

$$
p(\mathbf{x}, \mathbf{z}) = p(\mathbf{x} \:|\: \mathbf{z}) p(\mathbf{z})\;\;,
$$
y también podemos marginalizar sobre las variables latentes para recuperar el modelo anterior:

$$
p(\mathbf{x}) = \sum_\mathbf{z} p(\mathbf{x} \:|\: \mathbf{z}) p(\mathbf{z}) = \sum_{k=1}^K \pi_k{\mathcal{N}(\mathbf{x} \:|\: \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)}\;\;.
$$

O sea, estoy hace media hora dándole vueltas a las ecuaciones para llegar a lo mismo. ¿O no? 
En principio, vean que para cada dato $n$ habrá una variable latente $\mathbf{z}_n$.

__El algoritmo EM__

La gran ventaja de este planteo es que ahora podemos trabajar con la distribución conjunta $p(\mathbf{x}, \mathbf{z})$, que nos da pie para poner en marcha un algoritmo muy general, que sirva para todos los modelos que usen variables latentes. 

Acá vamos a presentarlo solo para las mixturas gaussianas. Pueden encontrar un tratamiento mucho más general en la Sect. 9.4 del Bishop.

Vamos a definir dos matrices, con los datos incompletos $\mathbf{X}$ y con las variables latentes $\mathbf{Z}$, cuya fila $n$ corresponde al vector de variables latentes del dato $\mathbf{x}_n$, $\mathbf{z}_n$.
Primero, volvamos a escribir el logaritmo de la verosimilitud:
$$
\ln p(\mathbf{X} \:|\: \boldsymbol{\mu}, \boldsymbol{\Sigma}, \boldsymbol{\pi}) = \sum_{n=1}^N \ln \left\{\sum_{k=1}^K \pi_k{\mathcal{N}(\mathbf{x}_n \:|\: \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)}\right\}\;\;.
$$

Derivando con respecto a $\boldsymbol{\mu}_k$ e igualando a cero, tenemos:

$$
0 = \sum_{n=1}^N \gamma(z_{nk}) \Sigma_k^{-1} (\mathbf{x_n} - \boldsymbol{\mu}_k)\;\;.
$$

Multiplicando a ambos lados por $\Sigma_k$, llegamos a:

$$
\boldsymbol{\mu}_k = \frac{1}{N_k} \sum_{n=1}^N {\gamma(z_{nk}) \mathbf{x}_n}\;\;,
$$

con

$$
N_k = \sum_{n=1}^N {\gamma(z_{nk})}\;\;,
$$

que es una especie de número efectivo de muestras provenientes del modo $k$.

Lo mismo podemos hacer para la covarianza $\Sigma_k$ y da

$$
\boldsymbol{\Sigma}_k = \frac{1}{N_k} \sum_{n=1}^N {\gamma(z_{nk}) (\mathbf{x}_n - \boldsymbol{\mu}_k)(\mathbf{x}_n - \boldsymbol{\mu}_k)^T}\;\;,
$$

y para los coeficientes de mezcla, usando el constrain (via multiplicadores de Lagrange) de que su suma tienen que ser uno.

$$
\pi_k = \frac{N_k}{N}\;\;.
$$

Hay que ver que la expresión para las medias y la covarianza tiene la misma pinta que para el caso de una sola distribución gaussiana. Pero como aparecen las responsabilidades, la forma no es cerrada.

Se vislumbra un proceso iterativo de la siguiente pinta:

1. Se inicializan los parámetros $\boldsymbol{\mu}_k$, $\boldsymbol{\Sigma}_k$, y $\pi_k$.

2. Se calculan las responsabilidades para esos valores de los parámetros (paso E).

3. Con las nuevas responsabilidades, se recalculan los parámetros que maximizan la verosimilitud (paso M). Como antes, podemos actualizar las medias antes de calcular el nuevo valor de las covarianzas.

4. Volver a calcular el logaritmo de la verosimilitud.

5. Evaluar algún criterio de detención, sea en LogL o en los parámetros.

Vamos a ver cómo funciona con nuestro set unidimensional. `sklearn` tiene implementado el algo EM en la clase `
mixture.GaussianMixture`

In [ ]:
from sklearn.mixture import GaussianMixture

# Veamos la docu
GaussianMixture?

In [ ]:
gm = GaussianMixture(2, means_init=np.array([0.0, 5.0]).reshape(2, 1), weights_init=[0.5, 0.5], tol=1e-8)
gm.fit(x)

print('Fitted parameters')
print('Mixing coeff:', gm.weights_)
print('Means:', gm.means_)
print('widhts:', np.sqrt(gm.covariances_))

# Supongamos una distribución unidimensional de dos modos, con los siguientes parámetros
# mu1 = [0.0,]
# sigma1 = [2.0,]
# mu2 = [5.0,]
# sigma2 = [1.5,]
# p1 = 0.65
# p2 = 0.35


In [ ]:
# Comparemos con la densidad real
plt.plot(xx, p1 * n1.pdf(xx) + p2 * n2.pdf(xx), 'r-', label='Truth', lw=5, alpha=0.4)
plt.plot(xx, np.exp(gm.score_samples(xx.reshape(-1, 1))), label='ML fit', color='k')
plt.legend(loc=0)

In [ ]:
tp = gm.predict(x)

***
**A trabajar!**

Ahora a ustedes. Tomen el dataset de Old Faithful de acá arriba y ajusten un modelo de mixtura. Usen la clase de skelearn.

Después copien el código que está arriba para plotear los contornos.

In [ ]:
gm = GaussianMixture(2, tol=1e-3)
gm.fit(df)

print('Fitted parameters')
print('Mixing coeff:', gm.weights_)
print('Means:', gm.means_)
print('widhts:', np.sqrt(gm.covariances_))

In [ ]:
# Primero construyo una grilla de valores
x, y = np.mgrid[1.0:6.0:0.1, 40:100:1]
pos = pos = np.dstack((x, y))

# Compute mixture pdf

# Usando sklearn
X_ = np.c_[x.ravel(), y.ravel()]

# Calculo la clase
clase1d = gm.predict(X_)
clase2d = clase1d.reshape(x.shape)

# Calculo la densidad
pdf = np.exp(gm.score_samples(X_))
P = pdf.reshape(x.shape)

# "A mano"
# P = (gm.weights_[0] * st.multivariate_normal.pdf(pos, mean=gm.means_[0], cov=gm.covariances_[0]) +
#        gm.weights_[1] * st.multivariate_normal.pdf(pos, mean=gm.means_[1], cov=gm.covariances_[1]))

In [ ]:
# Prepara figure
fig = plt.figure()
ax = fig.add_subplot(111)

# Vuelve a graficar datos y densidad
l0 = ax.contourf(x, y, P, 256, alpha=1)
df.plot('eruptions', 'waiting', kind='scatter', ax=ax, c='w', alpha=0.4)


In [ ]:
# Prepara figure
fig = plt.figure()
ax = fig.add_subplot(111)

# Vuelve a graficar datos y clases
l0 = ax.contourf(x, y, clase2d, alpha=1)
df.plot('eruptions', 'waiting', kind='scatter', ax=ax, c=gm.predict(df), alpha=0.8)

In [ ]:
# Predicción de clase para un nuevo dato y su responsabilidad.

x_nuevo = np.array([3.0, 65.0]).reshape(1, 2)
print(gm.predict(x_nuevo))
print(gm.predict_proba(x_nuevo))

In [ ]:
# Marginalizamos uno de los features (la duración de la erupción).

xx = np.linspace(40, 100, 200)

scale1 = np.sqrt(gm.covariances_[0][1,1])
scale2 = np.sqrt(gm.covariances_[1][1,1])

pp = (gm.weights_[0] * st.norm.pdf(xx, loc=gm.means_[0][1], scale=scale1) + 
      gm.weights_[1] * st.norm.pdf(xx, loc=gm.means_[1][1], scale=scale2))

plt.plot(xx, pp)
plt.xlabel('waiting')

In [ ]:
# Nos quedamos con una única componente
k = 1

# No vale la pena incluir el coeficiente, porque después hay que normalizar
pp = st.multivariate_normal.pdf(pos, mean=gm.means_[k], cov=gm.covariances_[k])

l0 = plt.contourf(x, y, pp, alpha=1)
#plt.colorbar(l0)
#df.plot('eruptions', 'waiting', kind='scatter', ax=ax, c='w', alpha=0.3)

### K-means

Es un algoritmo de clustering que funciona minimizando la distancia de los puntos a los centros de los clusters.

Veamos su funcionamiento con los datos de Old Faithful

In [ ]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=2)
km.fit(df)

In [ ]:
# Veamos los parámetros
km.cluster_centers_

In [ ]:
plt.scatter(df.eruptions, df.waiting, c=km.labels_)
plt.plot(km.cluster_centers_[:, 0], km.cluster_centers_[:, 1], 'or', markersize=10)